In [61]:
import pickle
import numpy as np
import json
import pandas as pd
import os

In [62]:
output_files = ['./output/12_test.bin', './output/13_test.bin']

In [63]:
logits = []

for i in output_files:
    with open(i, "rb") as f:
        logits.append(pickle.load(f).astype(np.uint16))

output = sum(logits)
output = output.argmax(axis=1)


In [64]:
# sample_submisson.csv 열기
submission = pd.read_csv('./output/sample_submission.csv', index_col=None)
json_dir = os.path.join("../input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('./output/', f'ensemble_softvoting.csv'), index=False)

  0%|          | 0/624 [01:41<?, ?it/s]
